In [12]:
import json
import logging
import os
import time

import mxnet as mx
from mxnet import autograd as ag
from mxnet import gluon
from mxnet.gluon.model_zoo import vision as models

## Downloading CIFAR10 dataset

In [13]:
from cifar10_utils import download_training_data
download_training_data()

downloading training data...
done


## Preparing Data

In [14]:
hyperparameters={'batch_size': 20, 
                           'epochs': 1, 
                           'learning_rate': 0.1, 
                           'momentum': 0.9}

def get_data(path, augment, num_cpus, batch_size, data_shape, resize=-1, num_parts=1, part_index=0):
    return mx.io.ImageRecordIter(
        path_imgrec=path,
        resize=resize,
        data_shape=data_shape,
        batch_size=batch_size,
        rand_crop=augment,
        rand_mirror=augment,
        preprocess_threads=num_cpus,
        num_parts=num_parts,
        part_index=part_index)


def get_test_data(num_cpus, data_dir, batch_size, data_shape, resize=-1):
    return get_data(os.path.join(data_dir, "test.rec"), False, num_cpus, batch_size, data_shape, resize, 1, 0)


def get_train_data(num_cpus, data_dir, batch_size, data_shape, resize=-1, num_parts=1, part_index=0):
    return get_data(os.path.join(data_dir, "train.rec"), True, num_cpus, batch_size, data_shape, resize, num_parts,
                    part_index)

def test(ctx, net, test_data):
    test_data.reset()
    metric = mx.metric.Accuracy()

    for i, batch in enumerate(test_data):
        data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
        label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
        outputs = []
        for x in data:
            outputs.append(net(x))
        metric.update(label, outputs)
    return metric.get()


## Loading Resnet model

In [15]:
hosts = [1]
part_index = 0
data_dir = "data"
model_dir = "model"
num_cpus = 2
batch_size = hyperparameters.get('batch_size', 128)
train_data = get_train_data(num_cpus, data_dir, batch_size, (3, 32, 32),
                            num_parts=len(hosts), part_index=part_index)
test_data = get_test_data(num_cpus, data_dir, batch_size, (3, 32, 32))

num_gpus = 1

batch_size = hyperparameters.get('batch_size', 128)
epochs = hyperparameters.get('epochs', 100)
learning_rate = hyperparameters.get('learning_rate', 0.1)
momentum = hyperparameters.get('momentum', 0.9)
log_interval = hyperparameters.get('log_interval', 1)
wd = hyperparameters.get('wd', 0.0001)

if len(hosts) == 1:
    kvstore = 'device' if num_gpus > 0 else 'local'
else:
    kvstore = 'dist_device_sync'

ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
net = models.get_model('resnet34_v2', ctx=ctx, pretrained=False, classes=10)
batch_size *= max(1, len(ctx))

## Transfer Learn the Network

In [16]:
    # Collect all parameters from net and its children, then initialize them.
    net.initialize(mx.init.Xavier(magnitude=2), ctx=ctx)
    # Trainer is for updating parameters with gradient.
    trainer = gluon.Trainer(net.collect_params(), 'sgd',
                            optimizer_params={'learning_rate': learning_rate, 'momentum': momentum, 'wd': wd},
                            kvstore=kvstore)
    metric = mx.metric.Accuracy()
    loss = gluon.loss.SoftmaxCrossEntropyLoss()

    best_accuracy = 0.0
    for epoch in range(epochs):
        # reset data iterator and metric at begining of epoch.
        train_data.reset()
        tic = time.time()
        metric.reset()
        btic = time.time()

        for i, batch in enumerate(train_data):
            data = gluon.utils.split_and_load(batch.data[0], ctx_list=ctx, batch_axis=0)
            label = gluon.utils.split_and_load(batch.label[0], ctx_list=ctx, batch_axis=0)
            outputs = []
            Ls = []
            with ag.record():
                for x, y in zip(data, label): 
                    z = net(x)
                    L = loss(z, y)
                    # store the loss and do backward after we have done forward
                    # on all GPUs for better speed on multiple GPUs.
                    Ls.append(L)
                    outputs.append(z)
                for L in Ls:
                    L.backward()
            trainer.step(batch.data[0].shape[0])
            metric.update(label, outputs)
            if i % log_interval == 0 and i > 0:
                name, acc = metric.get()
                print('Epoch [%d] Batch [%d]\tSpeed: %f samples/sec\t%s=%f' %
                             (epoch, i, batch_size / (time.time() - btic), name, acc))
            btic = time.time()
        name, acc = metric.get()
        print('[Epoch %d] training: %s=%f' % (epoch, name, acc))
        print('[Epoch %d] time cost: %f' % (epoch, time.time() - tic))

        name, val_acc = test(ctx, net, test_data)
        print('[Epoch %d] validation: %s=%f' % (epoch, name, val_acc))
        

Epoch [0] Batch [1]	Speed: 213.394657 samples/sec	accuracy=0.125000
Epoch [0] Batch [2]	Speed: 215.658755 samples/sec	accuracy=0.150000
Epoch [0] Batch [3]	Speed: 218.443094 samples/sec	accuracy=0.175000
Epoch [0] Batch [4]	Speed: 220.453489 samples/sec	accuracy=0.150000
Epoch [0] Batch [5]	Speed: 218.608181 samples/sec	accuracy=0.133333
Epoch [0] Batch [6]	Speed: 216.335611 samples/sec	accuracy=0.128571
Epoch [0] Batch [7]	Speed: 221.288010 samples/sec	accuracy=0.137500
Epoch [0] Batch [8]	Speed: 221.717312 samples/sec	accuracy=0.150000
Epoch [0] Batch [9]	Speed: 222.654783 samples/sec	accuracy=0.150000
Epoch [0] Batch [10]	Speed: 220.781892 samples/sec	accuracy=0.150000
Epoch [0] Batch [11]	Speed: 227.596304 samples/sec	accuracy=0.154167
Epoch [0] Batch [12]	Speed: 232.444172 samples/sec	accuracy=0.150000
Epoch [0] Batch [13]	Speed: 231.948637 samples/sec	accuracy=0.142857
Epoch [0] Batch [14]	Speed: 231.056121 samples/sec	accuracy=0.136667
Epoch [0] Batch [15]	Speed: 228.733224 samp

Epoch [0] Batch [121]	Speed: 235.607260 samples/sec	accuracy=0.104918
Epoch [0] Batch [122]	Speed: 231.887083 samples/sec	accuracy=0.104878
Epoch [0] Batch [123]	Speed: 231.865291 samples/sec	accuracy=0.104839
Epoch [0] Batch [124]	Speed: 232.560811 samples/sec	accuracy=0.104800
Epoch [0] Batch [125]	Speed: 231.561908 samples/sec	accuracy=0.105159
Epoch [0] Batch [126]	Speed: 228.948599 samples/sec	accuracy=0.104724
Epoch [0] Batch [127]	Speed: 234.534851 samples/sec	accuracy=0.105859
Epoch [0] Batch [128]	Speed: 229.996271 samples/sec	accuracy=0.105039
Epoch [0] Batch [129]	Speed: 233.948785 samples/sec	accuracy=0.104231
Epoch [0] Batch [130]	Speed: 232.749873 samples/sec	accuracy=0.103435
Epoch [0] Batch [131]	Speed: 233.037603 samples/sec	accuracy=0.104167
Epoch [0] Batch [132]	Speed: 235.496140 samples/sec	accuracy=0.104135
Epoch [0] Batch [133]	Speed: 235.910064 samples/sec	accuracy=0.103731
Epoch [0] Batch [134]	Speed: 232.088535 samples/sec	accuracy=0.103704
Epoch [0] Batch [135

Epoch [0] Batch [241]	Speed: 229.874630 samples/sec	accuracy=0.108678
Epoch [0] Batch [242]	Speed: 232.150195 samples/sec	accuracy=0.108436
Epoch [0] Batch [243]	Speed: 231.908239 samples/sec	accuracy=0.108402
Epoch [0] Batch [244]	Speed: 229.352348 samples/sec	accuracy=0.108367
Epoch [0] Batch [245]	Speed: 209.054067 samples/sec	accuracy=0.108943
Epoch [0] Batch [246]	Speed: 231.166274 samples/sec	accuracy=0.109109
Epoch [0] Batch [247]	Speed: 229.271484 samples/sec	accuracy=0.108871
Epoch [0] Batch [248]	Speed: 232.961884 samples/sec	accuracy=0.109237
Epoch [0] Batch [249]	Speed: 235.551687 samples/sec	accuracy=0.109200
Epoch [0] Batch [250]	Speed: 233.230684 samples/sec	accuracy=0.108765
Epoch [0] Batch [251]	Speed: 235.557640 samples/sec	accuracy=0.108730
Epoch [0] Batch [252]	Speed: 232.201603 samples/sec	accuracy=0.108498
Epoch [0] Batch [253]	Speed: 234.036247 samples/sec	accuracy=0.108071
Epoch [0] Batch [254]	Speed: 235.040193 samples/sec	accuracy=0.107843
Epoch [0] Batch [255

Epoch [0] Batch [361]	Speed: 231.422644 samples/sec	accuracy=0.115884
Epoch [0] Batch [362]	Speed: 234.389369 samples/sec	accuracy=0.116529
Epoch [0] Batch [363]	Speed: 232.612402 samples/sec	accuracy=0.116346
Epoch [0] Batch [364]	Speed: 235.269595 samples/sec	accuracy=0.116438
Epoch [0] Batch [365]	Speed: 235.089595 samples/sec	accuracy=0.116393
Epoch [0] Batch [366]	Speed: 235.915371 samples/sec	accuracy=0.116349
Epoch [0] Batch [367]	Speed: 233.219660 samples/sec	accuracy=0.116304
Epoch [0] Batch [368]	Speed: 235.069832 samples/sec	accuracy=0.116802
Epoch [0] Batch [369]	Speed: 232.360471 samples/sec	accuracy=0.116892
Epoch [0] Batch [370]	Speed: 230.866623 samples/sec	accuracy=0.116846
Epoch [0] Batch [371]	Speed: 236.459343 samples/sec	accuracy=0.116801
Epoch [0] Batch [372]	Speed: 238.834499 samples/sec	accuracy=0.116622
Epoch [0] Batch [373]	Speed: 228.099445 samples/sec	accuracy=0.116711
Epoch [0] Batch [374]	Speed: 231.905674 samples/sec	accuracy=0.116667
Epoch [0] Batch [375

Epoch [0] Batch [481]	Speed: 234.535506 samples/sec	accuracy=0.121162
Epoch [0] Batch [482]	Speed: 235.704576 samples/sec	accuracy=0.121429
Epoch [0] Batch [483]	Speed: 236.342757 samples/sec	accuracy=0.121281
Epoch [0] Batch [484]	Speed: 231.478841 samples/sec	accuracy=0.121443
Epoch [0] Batch [485]	Speed: 232.515044 samples/sec	accuracy=0.121605
Epoch [0] Batch [486]	Speed: 234.263692 samples/sec	accuracy=0.121869
Epoch [0] Batch [487]	Speed: 231.115323 samples/sec	accuracy=0.121926
Epoch [0] Batch [488]	Speed: 230.483491 samples/sec	accuracy=0.122086
Epoch [0] Batch [489]	Speed: 230.803102 samples/sec	accuracy=0.122347
Epoch [0] Batch [490]	Speed: 234.893426 samples/sec	accuracy=0.122200
Epoch [0] Batch [491]	Speed: 235.937932 samples/sec	accuracy=0.122256
Epoch [0] Batch [492]	Speed: 232.346955 samples/sec	accuracy=0.122008
Epoch [0] Batch [493]	Speed: 234.233602 samples/sec	accuracy=0.121862
Epoch [0] Batch [494]	Speed: 230.805643 samples/sec	accuracy=0.121717
Epoch [0] Batch [495

...

Epoch [0] Batch [2014]	Speed: 236.989759 samples/sec	accuracy=0.189007
Epoch [0] Batch [2015]	Speed: 230.505024 samples/sec	accuracy=0.188963
Epoch [0] Batch [2016]	Speed: 229.553186 samples/sec	accuracy=0.189043
Epoch [0] Batch [2017]	Speed: 230.632406 samples/sec	accuracy=0.189049
Epoch [0] Batch [2018]	Speed: 234.711375 samples/sec	accuracy=0.189054
Epoch [0] Batch [2019]	Speed: 232.847428 samples/sec	accuracy=0.189134
Epoch [0] Batch [2020]	Speed: 233.940303 samples/sec	accuracy=0.189213
Epoch [0] Batch [2021]	Speed: 228.660275 samples/sec	accuracy=0.189268
Epoch [0] Batch [2022]	Speed: 235.999662 samples/sec	accuracy=0.189372
Epoch [0] Batch [2023]	Speed: 234.390024 samples/sec	accuracy=0.189377
Epoch [0] Batch [2024]	Speed: 231.719901 samples/sec	accuracy=0.189432
Epoch [0] Batch [2025]	Speed: 236.938886 samples/sec	accuracy=0.189437
Epoch [0] Batch [2026]	Speed: 232.449969 samples/sec	accuracy=0.189492
Epoch [0] Batch [2027]	Speed: 234.510591 samples/sec	accuracy=0.189522
Epoch 

Epoch [0] Batch [2131]	Speed: 235.507379 samples/sec	accuracy=0.191628
Epoch [0] Batch [2132]	Speed: 236.175074 samples/sec	accuracy=0.191702
Epoch [0] Batch [2133]	Speed: 235.238587 samples/sec	accuracy=0.191706
Epoch [0] Batch [2134]	Speed: 234.568298 samples/sec	accuracy=0.191733
Epoch [0] Batch [2135]	Speed: 236.725590 samples/sec	accuracy=0.191760
Epoch [0] Batch [2136]	Speed: 232.260751 samples/sec	accuracy=0.191764
Epoch [0] Batch [2137]	Speed: 234.990153 samples/sec	accuracy=0.191862
Epoch [0] Batch [2138]	Speed: 237.563592 samples/sec	accuracy=0.191795
Epoch [0] Batch [2139]	Speed: 236.560700 samples/sec	accuracy=0.191822
Epoch [0] Batch [2140]	Speed: 235.515314 samples/sec	accuracy=0.191780
Epoch [0] Batch [2141]	Speed: 234.794150 samples/sec	accuracy=0.191853
Epoch [0] Batch [2142]	Speed: 232.080187 samples/sec	accuracy=0.191857
Epoch [0] Batch [2143]	Speed: 232.956061 samples/sec	accuracy=0.191908
Epoch [0] Batch [2144]	Speed: 233.084224 samples/sec	accuracy=0.191935
Epoch 

Epoch [0] Batch [2248]	Speed: 230.603242 samples/sec	accuracy=0.193886
Epoch [0] Batch [2249]	Speed: 232.152765 samples/sec	accuracy=0.193911
Epoch [0] Batch [2250]	Speed: 235.662859 samples/sec	accuracy=0.193980
Epoch [0] Batch [2251]	Speed: 234.170177 samples/sec	accuracy=0.193961
Epoch [0] Batch [2252]	Speed: 232.441596 samples/sec	accuracy=0.193964
Epoch [0] Batch [2253]	Speed: 234.913160 samples/sec	accuracy=0.193922
Epoch [0] Batch [2254]	Speed: 235.227373 samples/sec	accuracy=0.193902
Epoch [0] Batch [2255]	Speed: 232.168828 samples/sec	accuracy=0.193883
Epoch [0] Batch [2256]	Speed: 235.286093 samples/sec	accuracy=0.193930
Epoch [0] Batch [2257]	Speed: 231.053575 samples/sec	accuracy=0.194021
Epoch [0] Batch [2258]	Speed: 235.535152 samples/sec	accuracy=0.194135
Epoch [0] Batch [2259]	Speed: 236.831855 samples/sec	accuracy=0.194181
Epoch [0] Batch [2260]	Speed: 232.709841 samples/sec	accuracy=0.194206
Epoch [0] Batch [2261]	Speed: 231.379238 samples/sec	accuracy=0.194120
Epoch 

Epoch [0] Batch [2365]	Speed: 231.591316 samples/sec	accuracy=0.197168
Epoch [0] Batch [2366]	Speed: 234.306878 samples/sec	accuracy=0.197106
Epoch [0] Batch [2367]	Speed: 236.733607 samples/sec	accuracy=0.197044
Epoch [0] Batch [2368]	Speed: 231.851833 samples/sec	accuracy=0.197108
Epoch [0] Batch [2369]	Speed: 235.183847 samples/sec	accuracy=0.197110
Epoch [0] Batch [2370]	Speed: 231.607301 samples/sec	accuracy=0.197111
Epoch [0] Batch [2371]	Speed: 232.866819 samples/sec	accuracy=0.197133
Epoch [0] Batch [2372]	Speed: 233.972928 samples/sec	accuracy=0.197134
Epoch [0] Batch [2373]	Speed: 236.230942 samples/sec	accuracy=0.197178
Epoch [0] Batch [2374]	Speed: 234.227716 samples/sec	accuracy=0.197263
Epoch [0] Batch [2375]	Speed: 238.760403 samples/sec	accuracy=0.197222
Epoch [0] Batch [2376]	Speed: 234.249300 samples/sec	accuracy=0.197244
Epoch [0] Batch [2377]	Speed: 231.448823 samples/sec	accuracy=0.197246
Epoch [0] Batch [2378]	Speed: 233.189838 samples/sec	accuracy=0.197247
Epoch 

Epoch [0] Batch [2482]	Speed: 234.213983 samples/sec	accuracy=0.199678
Epoch [0] Batch [2483]	Speed: 237.431129 samples/sec	accuracy=0.199698
Epoch [0] Batch [2484]	Speed: 229.373670 samples/sec	accuracy=0.199698
Epoch [0] Batch [2485]	Speed: 230.909837 samples/sec	accuracy=0.199759
Epoch [0] Batch [2486]	Speed: 234.874354 samples/sec	accuracy=0.199839
Epoch [0] Batch [2487]	Speed: 237.933521 samples/sec	accuracy=0.199839
Epoch [0] Batch [2488]	Speed: 234.139458 samples/sec	accuracy=0.199879
Epoch [0] Batch [2489]	Speed: 235.200332 samples/sec	accuracy=0.199940
Epoch [0] Batch [2490]	Speed: 236.790407 samples/sec	accuracy=0.199960
Epoch [0] Batch [2491]	Speed: 233.991855 samples/sec	accuracy=0.199960
Epoch [0] Batch [2492]	Speed: 229.342315 samples/sec	accuracy=0.199960
Epoch [0] Batch [2493]	Speed: 236.353411 samples/sec	accuracy=0.199900
Epoch [0] Batch [2494]	Speed: 236.113916 samples/sec	accuracy=0.199940
Epoch [0] Batch [2495]	Speed: 235.757571 samples/sec	accuracy=0.199900
Epoch 

In [6]:
# save params
net.save_params('{}/model-{:0>4}.params'.format(model_dir, epoch))


### CIFAR10 sample images

We'll use these CIFAR10 sample images to test the service:

<img style="display: inline; height: 32px; margin: 0.25em" src="images/airplane1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/automobile1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/bird1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/cat1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/deer1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/dog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/frog1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/horse1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/ship1.png" />
<img style="display: inline; height: 32px; margin: 0.25em" src="images/truck1.png" />

In [27]:
net = models.get_model('resnet34_v2', ctx=mx.cpu(), pretrained=False, classes=10)
net.load_params('{}/model-{:0>4}.params'.format(model_dir, 0), ctx=mx.gpu(0))

In [30]:
# load the CIFAR10 samples, and convert them into format we can use with the prediction endpoint
from cifar10_utils import read_images

filenames = ['images/airplane1.png',
             'images/automobile1.png',
             'images/bird1.png',
             'images/cat1.png',
             'images/deer1.png',
             'images/dog1.png',
             'images/frog1.png',
             'images/horse1.png',
             'images/ship1.png',
             'images/truck1.png']

image_data = read_images(filenames)


3072


In [33]:
data = gluon.utils.split_and_load(image_data[0], ctx_list=[mx.gpu(0)], batch_axis=0)
for x in data:
    y = net(x)
    print(y)


[[ 1.38848114  1.2084254  -0.32151598 -1.30662465 -0.43435436 -1.46848369
  -1.27514076 -0.7148152   1.50975513  1.62946868]]
<NDArray 1x10 @gpu(0)>


In [34]:
test_data = get_test_data(num_cpus, data_dir, 1, (3, 32, 32))
for i, batch in enumerate(test_data):    
    data = gluon.utils.split_and_load(batch.data[0], ctx_list=[mx.gpu(0)], batch_axis=0)
    for x in data:
        y = net(x)
        prediction = mx.nd.argmax(y, axis=1)
        print(prediction)
        break
    break
    


[ 5.]
<NDArray 1 @gpu(0)>
